# Using grlc from python
Being written in python itself, it is easy to use grlc from python. Here we show how to use grlc to run a SPARQL query which is stored on github.

First we start by importing grlc (and a couple other libraries we use for working with the data).

In [15]:
import json
import pandas as pd
from io import StringIO

import grlc
import grlc.utils as utils
import grlc.swagger as swagger

We can load the grlc specification for a github repository. For example, the grlc test query repository https://github.com/CLARIAH/grlc-queries.

In [ ]:
user = 'CLARIAH'
repo = 'grlc-queries'
spec, warning = swagger.build_spec(user, repo)

We can have a look at one item of the returned specification.

In [18]:
print(json.dumps(spec[0], indent=2))

{
  "call_name": "/description",
  "method": "get",
  "tags": [],
  "summary": "",
  "description": "Extended description of my query/operation. This can be a more extensive than the summary and in can span multiple lines.\nDescriptions support (to some extent) [CommonMark](https://commonmark.org/help/).",
  "params": [
    {
      "name": "endpoint",
      "type": "string",
      "in": "query",
      "description": "Alternative endpoint for SPARQL query",
      "default": "https://dbpedia.org/sparql"
    }
  ],
  "item_properties": null,
  "query": "\nPREFIX dbo: <http://dbpedia.org/ontology/>\nPREFIX schema: <http://schema.org/>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n\nSELECT ?band ?album ?genre WHERE {\n  ?band rdf:type dbo:Band .\n  ?album rdf:type schema:MusicAlbum .\n  ?band dbo:genre ?genre .\n  ?album dbo:artist ?band .\n} LIMIT 100\n",
  "original_query": "#+ description: Extended description of my query/operation.\n#+     This can be a more extensive than

This specification item will execute the following SPARQL query:

In [4]:
print(spec[0]['query'])


PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?band ?album ?genre WHERE {
  ?band rdf:type dbo:Band .
  ?album rdf:type schema:MusicAlbum .
  ?band dbo:genre ?genre .
  ?album dbo:artist ?band .
} LIMIT 100



We can execute this query by calling the API point in our generated API:

In [5]:
print(spec[0]['call_name'])

/description


We can use `dispatch_query` functions to load data from a specific query (`description` in this case). For this example, we are loading data in `text/csv` format.

**NOTE:** `description` query loads data from dbpedia.org -- the endpoint is specified in the repository `endpoint.txt` file.

In [6]:
query_name = 'description'
acceptHeader = 'text/csv'

data, code, headers = utils.dispatch_query(user, repo, query_name, acceptHeader=acceptHeader)

Requesting query at https://raw.githubusercontent.com/CLARIAH/grlc-queries/master/description.rq
Requesting query at https://raw.githubusercontent.com/CLARIAH/grlc-queries/master/endpoint.txt


Now we just transform these results to a pandas dataframe.

In [7]:
data_grlc = pd.read_csv(StringIO(data))
data_grlc.head(10)

,band,album,genre
0,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Art_rock
1,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Progressive_rock
2,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Album-oriented_rock
3,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Arena_rock
4,http://dbpedia.org/resource/Bauhaus_(band),http://dbpedia.org/resource/Swing_the_Heartach...,http://dbpedia.org/resource/Gothic_rock
5,http://dbpedia.org/resource/Bauhaus_(band),http://dbpedia.org/resource/Swing_the_Heartach...,http://dbpedia.org/resource/Post-punk
6,http://dbpedia.org/resource/Catatonia_(band),http://dbpedia.org/resource/Paper_Scissors_Sto...,http://dbpedia.org/resource/Alternative_rock
7,http://dbpedia.org/resource/Catatonia_(band),http://dbpedia.org/resource/Paper_Scissors_Sto...,http://dbpedia.org/resource/Indie_rock
8,http://dbpedia.org/resource/Siouxsie_and_the_B...,http://dbpedia.org/resource/Tinderbox_(Siouxsi...,http://dbpedia.org/resource/Gothic_rock
9,http://dbpedia.org/resource/Siouxsie_and_the_B...,http://dbpedia.org/resource/Tinderbox_(Siouxsi...,http://dbpedia.org/resource/New_wave_music


# Grlc via http

Another alternative is to load data via a running grlc server (in this case `grlc.io`).

In [8]:
import requests

In [9]:
headers = {'accept': 'text/csv'}
resp = requests.get("http://grlc.io/api-git/CLARIAH/grlc-queries/description", headers=headers)

In [10]:
data_requests = pd.read_csv(StringIO(resp.text))
data_requests.head(10)

,band,album,genre
0,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Art_rock
1,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Progressive_rock
2,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Album-oriented_rock
3,http://dbpedia.org/resource/Asia_(band),http://dbpedia.org/resource/Axioms_(album),http://dbpedia.org/resource/Arena_rock
4,http://dbpedia.org/resource/Bauhaus_(band),http://dbpedia.org/resource/Swing_the_Heartach...,http://dbpedia.org/resource/Gothic_rock
5,http://dbpedia.org/resource/Bauhaus_(band),http://dbpedia.org/resource/Swing_the_Heartach...,http://dbpedia.org/resource/Post-punk
6,http://dbpedia.org/resource/Catatonia_(band),http://dbpedia.org/resource/Paper_Scissors_Sto...,http://dbpedia.org/resource/Alternative_rock
7,http://dbpedia.org/resource/Catatonia_(band),http://dbpedia.org/resource/Paper_Scissors_Sto...,http://dbpedia.org/resource/Indie_rock
8,http://dbpedia.org/resource/Siouxsie_and_the_B...,http://dbpedia.org/resource/Tinderbox_(Siouxsi...,http://dbpedia.org/resource/Gothic_rock
9,http://dbpedia.org/resource/Siouxsie_and_the_B...,http://dbpedia.org/resource/Tinderbox_(Siouxsi...,http://dbpedia.org/resource/New_wave_music
